### Customer Status Process

In [1]:
import pandas as pd

#Reading the required excel sheets and columns from the BI Daily Reports. The selected columns are considered valid for the required task
r1 = pd.read_excel('BI NG daily reports of 2022-03-31.xlsx','Contracts')

r2 = pd.read_excel('BI NG daily reports of 2022-03-31.xlsx','Sps Contracts')

r3 = pd.read_excel('BI NG daily reports of 2022-03-31.xlsx','Contracts per PaidPeriod')

r4 = pd.read_excel('BI NG daily reports of 2022-03-31.xlsx','CRM contracts', names = ['Contract','CRMcontract'])

r5 = pd.read_excel('BI NG daily reports of 2022-03-31.xlsx','Last payments', usecols = ['Contract','payment_date','activation_end'])

r5.columns = ['Contract','last_payment_date','lastpayment_activation_end']

big_table = pd.read_csv("Bigtable_Mar'22.csv", usecols = ['Full SFID','Type','Status','StatusII','Product Name','Customer Status'], encoding='windows-1252')

big_table.columns = ['Contract','Type','Status','StatusII','Product Name','Customer Status']

#Selecting specific columns.....
#custom = pd.read_csv('CRM_Big_Table.csv',usecols = ['Full SFID','Contract Id','Type','Status'], encoding='windows-1252')

In [2]:
r1['ContractCreation'] = pd.to_datetime(r1['ContractCreation']).dt.normalize()
r2['activation_start'] = pd.to_datetime(r2['activation_start']).dt.normalize()
r2['activation_end'] = pd.to_datetime(r2['activation_end']).dt.normalize()
r5['last_payment_date'] = pd.to_datetime(r5['last_payment_date']).dt.normalize()
r5['lastpayment_activation_end'] = pd.to_datetime(r5['lastpayment_activation_end']).dt.normalize()

In [3]:
check1 = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(r1, r2, how='left', on='Contract'), r3, how='left', on='Contract'), r4, how='left', on='Contract'), r5, how='left', on='Contract'), big_table, how='left', on='Contract') 

check1 = check1[check1['Type'] == 'Production']

In [4]:
#join1.isnull().sum().sort_values(ascending = False)
check1["ref_activation_end"] = check1[["activation_end","lastpayment_activation_end"]].max(axis=1)

check1['ref_activation_end'] = check1['ref_activation_end'].fillna('1900-01-01')

#check1['outage_days'] = (check1['ref_activation_end'] - to_day).dt.days

In [5]:
import datetime as date
from datetime import date
from datetime import timedelta

import numpy as np

today = pd.to_datetime('today').normalize()

to_day = today - timedelta(days=4)

check1['outage_days'] = (check1['ref_activation_end'] - to_day).dt.days

TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'

In [ ]:
#Creates an extra column to indicate the active status of a contract by manipulating the Outage Days
import numpy as np

check1["old_active_status"] = np.where(
   (check1.outage_days > -60), 
   "Active", 
   "Churn"
    )

In [ ]:
#Creates an extra column to indicate the STATUS of a contract by manipulating the Outage Days
check1['new_active_status'] = (
    np.where(
        check1['outage_days'] > -11, 
        'Paid', 
        np.where(check1['outage_days'] < -40, 
                 'Retrieval',
                np.where(check1['outage_days'] < -10, 
                        'Recovery', 'Unavailable'))))


check1['Product Name'] = check1['Product Name'].fillna('null')

In [ ]:
#Creates an extra column to indicate the LIGHT status of a contract by manipulating the Outage Days
check1["light_status"] = np.where(
   (check1.outage_days >= 0),
   "InLight", 
   "InDark"
    )

In [ ]:
#Creates an extra column which derives the age of a contract since inception
check1["age"] = np.where(
   (check1.LTOunits == 'MONTHS'),
   (to_day - check1['ContractCreation']).dt.days/30.4, 
   (to_day - check1['ContractCreation']).dt.days
    )

In [ ]:
import numpy as np

check1["segmentation_score"] = (check1['PaidPeriods']/check1['age'])*100


filters = [
   (check1.PaidPeriods > 0) & (check1.age == 0)
]
values = [100]

check1["segmentation_score"] = np.select(filters, values, default = check1.segmentation_score)


check1.replace([np.inf, -np.inf], np.nan,  inplace=True)

In [ ]:
check1["lto_segment"] = np.where(
   (check1.LTOperiod >= check1.age), 
   "Within_LTO", 
   "Outside_LTO"
)

#check1.head()

In [ ]:
filters = [
   (check1.segmentation_score >= 100),
   (check1.segmentation_score < 100) & (check1.segmentation_score >= 80),
   (check1.segmentation_score < 80) & (check1.segmentation_score >= 41)
]
values = ['Good', 'Fair', 'Challenging']

check1["segmentation_status"] = np.select(filters, values, default = 'Bad')

In [ ]:
filters = [
   (check1['Product Name'].str.contains("NWO", case=False))
]
values = ['LCP']

check1["lcp_status"] = np.select(filters, values, default='Non-LCP')

#check1.head()

In [ ]:
filters = [
   (check1.generation == 'LEGACY') & ((check1.age/30.4) >= 5),
   (check1.generation == 'UNIFIED') & (check1.age >= 5)
]
values = ['Pre-Existing','Pre-Existing']

check1["age_segment"] = np.select(filters, values, default = 'New')

#check1.head()

In [ ]:
filters = [
   (check1.Status == 'Cancelled') | (check1.Status == 'Canceled'),
   (check1.StatusII == 'Free') | (check1.StatusII == 'Freeing'),
   (check1['Customer Status'] == 'Owner'),
   (check1.FreeStatus == 'FREE')
   #(check1.PaidPeriods == check1.LTOperiod)
]
values = ['Cancelled','Owner','Owner','Owner']

check1["customer_status"] = np.select(filters, values, default= check1.new_active_status)

#check1.head()

In [ ]:
check1['FreeStatus'] = check1['FreeStatus'].fillna('null')
check1['SpsId'] = check1['SpsId'].fillna(0)
check1['StatusII'] = check1['StatusII'].fillna('null')
check1['Customer Status'] = check1['Customer Status'].fillna('null')
check1['segmentation_score'] = check1['segmentation_score'].fillna(0)
check1['PaidPeriods'] = check1['PaidPeriods'].fillna(0)
check1['lastpayment_activation_end'] = check1['lastpayment_activation_end'].fillna('1900-01-01')
check1['last_payment_date'] = check1['last_payment_date'].fillna('1900-01-01')
check1['activation_start'] = check1['activation_start'].fillna('1900-01-01')
check1['activation_end'] = check1['activation_end'].fillna('1900-01-01')

In [ ]:
filters = [
   (check1.customer_status == 'Cancelled'),
   (check1.customer_status == 'Owner')
   #(check1.PaidPeriods == check1.LTOperiod)
]
values = ['Cancelled','Owner']

check1["old_active_status"] = np.select(filters, values, default= check1.old_active_status)

In [ ]:
check1 = check1.drop(['LTOunits','activation_end','FreeStatus','Type','Status','StatusII','new_active_status','Customer Status','lastpayment_activation_end'], axis=1)

In [ ]:
#check1.groupby(['old_active_status']).count()['Contract']
#check1.info()

In [ ]:
import mysql.connector as msql
from mysql.connector import Error

try:
    conn = msql.connect(host='localhost', 
                           database='lumos_bi_ng', user='root', 
                           password='', port='3306')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to lumos_bi_ng database: ", record)
        cursor.execute('DROP TABLE IF EXISTS activity_mar22;')
        #print('Creating table....')
        cursor.execute("CREATE TABLE activity_mar22 (contract VARCHAR(50) NULL,reportdate DATE NULL,sales_channel VARCHAR(20) NULL,generation VARCHAR(20) NULL,type VARCHAR(20) NULL,ltoperiod BIGINT NULL,contract_creation DATE NULL,spsid BIGINT NULL,activation_start DATE NULL,paidperiod BIGINT NULL,crmcontract BIGINT NULL,last_payment_date DATE NULL,product_name VARCHAR(100) NULL,ref_activation_end DATE NULL,outage_days BIGINT NULL,old_active_status VARCHAR(15) NULL,light_status VARCHAR(10) NULL,age FLOAT NULL,segmentation_score FLOAT NULL,lto_segment VARCHAR(20) NULL,segmentation_status VARCHAR(20) NULL,lcp_status VARCHAR(20) NULL,age_segment VARCHAR(20) NULL,customer_status VARCHAR(20) NULL)")
        print("activity_mar22 is created....")
        for i,row in check1.iterrows():
            sql = "INSERT INTO lumos_bi_ng.activity_mar22 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            chunksize = 10000
            #print("Record inserted")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
except Error as e:
    print("Error while connecting to MySQL", e)
    
print("activity table completely loaded")

In [ ]:
import numpy as np

print(np.__version__)

In [ ]:
arr = np.array([[1, 2, 3], [4, 5, 6], [1, 2, 3], [4, 5, 6]])

print(arr)

type(arr)

In [ ]:
print(arr.ndim)

In [ ]:
d = np.array([[[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]])
print(d.ndim)

In [ ]:
arr = np.array([1, 2, 3, 4],  ndmin=4)

arr

In [ ]:
type(arr)

In [ ]:
np.zeros

In [ ]:
#Dropping a column
#check1 = check1.drop(['lastpayment_activation_end'], axis=1)

In [ ]:
#check1.isnull().sum().sort_values(ascending = False)
#check1.to_excel('check.xlsx')

In [ ]:
check1.groupby(['customer_status']).count()['Contract']

In [ ]:
#import pandas as pd
#print(pd.to_datetime('2022-03-31'))#.normalize()
type(check1['ref_activation_end'])

In [ ]:
check1['outage_days'] = (np.array(check1['ref_activation_end']) - np.array(to_day)).dt.days

In [ ]:
to_day = today - timedelta(days=4)